# IMPORTS

In [1]:
from theano.sandbox import cuda

In [2]:
# import file of useful functions
import os
import sys
from imp import reload
scriptpath = 'C:/Users/lyncp010/projects/python/scripts/'
sys.path.insert(0, os.path.abspath(scriptpath))

import deep_learning_utils; reload(deep_learning_utils)
from deep_learning_utils import *
from __future__ import division, print_function

%matplotlib inline

Using TensorFlow backend.


In [3]:
path = 'C:/Users/lyncp010/projects/nfl/data'

# DATA

In [4]:
offense_player_ff_points = pd.read_csv(path + '/all/offense_player_ff_points_s02w01_s16w21.csv', low_memory=False)

In [5]:
# use only players from current season
df = offense_player_ff_points[(offense_player_ff_points.season == 2016)].copy()

In [6]:
def create_category_idx(df, cols):
    map2idx = {}
    for c in cols:
        c_list = np.sort(df[c].unique())
        map2idx[c] = {o:i for i,o in enumerate(c_list)}
        df[c+'_idx'] = df[c].map(map2idx[c])
    return df, map2idx

In [7]:
df, map2idx = create_category_idx(df, ['player_id', 'team', 'opponent', 'position'])

In [22]:
df.head(10)

,season,week,bsID,team,player_id,position,ff_points_total,home,opponent,passAtt,...,rushLong,fumbles,fumblesLost,passYdsOver300,rushYdsOver100,recYdsOver100,player_id_idx,team_idx,opponent_idx,position_idx
71153,2016,1,201609080den,car,BenjKe00,WR,15.10,0,den,0.0,...,0.0,0.0,0.0,0,0,0,36,2,9,3
71154,2016,1,201609080den,car,BrowPh00,WR,1.10,0,den,0.0,...,0.0,0.0,0.0,0,0,0,68,2,9,3
71155,2016,1,201609080den,car,FuncDe00,WR,0.90,0,den,0.0,...,0.0,0.0,0.0,0,0,0,175,2,9,3
71156,2016,1,201609080den,car,GinnTe00,WR,2.50,0,den,0.0,...,20.0,0.0,0.0,0,0,0,183,2,9,3
71157,2016,1,201609080den,car,NewtCa00,QB,22.16,0,den,33.0,...,12.0,0.0,0.0,0,0,0,361,2,9,0
71158,2016,1,201609080den,car,OlseGr00,TE,7.30,0,den,0.0,...,0.0,0.0,0.0,0,0,0,368,2,9,2
71159,2016,1,201609080den,car,StewJo00,RB,6.40,0,den,0.0,...,11.0,0.0,0.0,0,0,0,461,2,9,1
71160,2016,1,201609080den,car,TolbMi00,RB,0.70,0,den,0.0,...,5.0,0.0,0.0,0,0,0,491,2,9,1
71161,2016,1,201609080den,car,WhitFo00,RB,1.70,0,den,0.0,...,6.0,0.0,0.0,0,0,0,526,2,9,1
71162,2016,1,201609080den,den,AndeC.00,RB,25.90,1,car,0.0,...,28.0,0.0,0.0,0,0,0,14,9,2,1


# MODEL

In [10]:
player_min, player_max, opponent_min, opponent_max = (
    df.player_id_idx.min(), 
    df.player_id_idx.max(),
    df.opponent_idx.min(),
    df.opponent_idx.max())
player_min, player_max, opponent_min, opponent_max

(0, 556, 0, 31)

In [11]:
n_players = df.player_id_idx.nunique()
n_teams = df.team_idx.nunique()
n_opponents = df.opponent_idx.nunique()
n_players, n_teams, n_opponents

(557, 32, 32)

number of latent factors

In [12]:
from keras.layers.merge import concatenate

In [13]:
foo = df.copy()

In [14]:
msk = foo.week <= 17

In [15]:
trn = foo[msk]
val = foo[~msk]

In [16]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, embeddings_regularizer=l2(reg))(inp)

In [17]:
n_factors = 20

In [18]:
player_in, p   = embedding_input('player_in', n_players, 7, 1e-4)
team_in, t     = embedding_input('team_in', n_teams, n_factors, 1e-4)
opponent_in, o = embedding_input('opponent_in', n_opponents, n_factors, 1e-4)

home_input = Input(shape=(1,), name='home_input')
position_input = Input(shape=(1,), name='position_input')

In [19]:
x = concatenate([p, t, o])
x = Flatten()(x)
x = concatenate([x, home_input, position_input])
x = Dropout(0.2)(x)
x = Dense(200, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(1)(x)
nn = Model([player_in, team_in, opponent_in, home_input, position_input], x)
nn.compile(Adam(0.001), loss='mse')

In [20]:
nn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
player_in (InputLayer)           (None, 1)             0                                            
____________________________________________________________________________________________________
team_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
opponent_in (InputLayer)         (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1, 7)          3899                                         
___________________________________________________________________________________________

# TRAIN

In [21]:
nn.fit([trn.player_id_idx, trn.team_idx, trn.opponent_idx, trn.home, trn.position_idx],
       trn.ff_points_total, batch_size=64, epochs=100,
       validation_data=([val.player_id_idx, val.team_idx, val.opponent_idx, val.home, val.position_idx], val.ff_points_total)
      )

Train on 5106 samples, validate on 214 samples
Epoch 1/100
5106/5106 [==============================] - 0s - loss: 70.4351 - val_loss: 66.1663
Epoch 2/100
5106/5106 [==============================] - 0s - loss: 44.9780 - val_loss: 50.3216
Epoch 3/100
5106/5106 [==============================] - 0s - loss: 33.7757 - val_loss: 39.8650
Epoch 4/100
5106/5106 [==============================] - 0s - loss: 27.8525 - val_loss: 35.7299
Epoch 5/100
5106/5106 [==============================] - 0s - loss: 25.6809 - val_loss: 34.8167
Epoch 6/100
5106/5106 [==============================] - 0s - loss: 25.1710 - val_loss: 34.1176
Epoch 7/100
5106/5106 [==============================] - 0s - loss: 25.1292 - val_loss: 33.2588
Epoch 8/100
5106/5106 [==============================] - 0s - loss: 24.8295 - val_loss: 33.1379
Epoch 9/100
5106/5106 [==============================] - 0s - loss: 24.3457 - val_loss: 33.1274
Epoch 10/100
5106/5106 [==============================] - 0s - loss: 24.3675 - val_loss: 